In [3]:
import os
from os.path import expanduser
import socket
import sys


# BELOW IS SYSTEM/PATH CONFIGURATION #
my_home = expanduser("~")
running_system = socket.gethostname()
# Configure machine's names here #
systemHLRN = 'blogin1'
systemLOCAL = 'Laurents-MacBook-Pro.local'
my_home = expanduser("~")
# Define local pass #
if running_system == systemLOCAL:
    print "computer = ", running_system
    wd_path = os.path.join(my_home,"Documents","projects", "awi-models")
elif running_system[0:1] == "b" or running_system == systemHLRN:
    print "computer = ", running_system
    wd_path = os.path.join(my_home,"awi-models")
else:
    print 'please configure your local machine : type socket.gethostname()'
print "working directory set to", wd_path
os.chdir(wd_path)
# IMPORT PY-FESOM MODULES
sys.path.append("codes/modules")
sys.path.append("codes/pyfesom-master/") # add pyfesom to search path
#########################################

computer =  bfn1001
working directory set to /home/hbkoziel/awi-models


In [4]:
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from pylab import *
from load_mesh_data_new import *
import numpy as np
import datetime as dt
from PiecewiseNorm import PiecewiseNorm
from netCDF4 import Dataset
#import colormaps as cmaps
from matplotlib.colors import ListedColormap
import pyfesom as pf

In [5]:
# PLOT CONFIGURATION #

# load colormaps #
cmap = plt.get_cmap('RdBu_r')
# figure export 
export_plot = True
# figure export definition
dpicnt=150
# choose simulation
runid='Arc12'
# choose date
first_year = 1990
last_year  = 2015
years      = np.arange(first_year,last_year+1,1)
months =np.linspace(0,11,12).astype(int)
# choose depth
get3d = True
# load the given biological tracer #
var_id1, var_id2 = 'tr06', 'tr15'
# export CSV
export_csv = True
########################

# AUTOMATIC DEFINITION OF PATHS
resultpath = '/scratch/usr/hbkvsk12/hlrn3_work2/results/'+runid+'/Oldfiles/'
savepath    = wd_path+'/figures/'+runid+'/'
meshpath    = wd_path+'/data/mesh/meshArc4.5/'
outputpath = '/scratch/usr/hbkoziel/Arc12/'

# Create figure directory if it does not exist
if os.path.exists(savepath) == False:
    try:
        os.mkdir(savepath)
        print ("Creation of the directory %s successfull" % savepath)
    except OSError:
        print ("Creation of the directory %s failed" % savepath)
else:
    print ("directory %s already existing" % savepath)

directory /home/hbkoziel/awi-models/figures/Arc12/ already existing


In [ ]:
mesh = fesom_mesh(meshpath, get3d = get3d)
#mesh = load_mesh(meshpath, usepickle=True,get3d=True)
mesh.n32 = mesh.n32-1
mesh.zlevs = -mesh.zlevs # depth is made negative

Load 2d part of the grid in 2 second(s)


In [ ]:
#ncfile      = resultpath+runid+'.initial.mesh.diag.nc'
#f           = Dataset(ncfile, 'r')
#NodalArea = f.variables['cluster_area'][:]
#NodalVol3D    = f.variables['cluster_vol'][:]
#f.close()
mesh

In [ ]:
mesh.zlevs

In [32]:
CHL  = np.zeros((len(years),len(mesh.x2)))
for mo in range(4,9):
    print mo
    for ye in range(0,len(years)):
     print years[ye]
     
     if years[ye] < 2000:
        ncfile  = resultpath+runid+'.'+str(years[ye])+'.oce.mean.nc'
        f       = Dataset(ncfile, 'r')    
        chl1     = f.variables[var_id1][mo,:]
        chl2     = f.variables[var_id2][mo,:]
        chl=chl1+chl2
        
     else:
        if ind ==4: 
         dayind = (range(45,59))
         month='APR'
        elif ind ==5: 
         dayind = (range(60,74))
         month='MAY'
        elif ind ==6: 
         dayind = (range(75,89))
         month='JUN'
        elif ind ==7: 
         dayind = (range(90,105))
         month='JUL'
        elif ind ==8:
         dayind = (range(106,120))
         month='AUG'
        elif ind ==9: 
         dayind = (range(121,135))
         month='SEP'
            
        ncfile  = resultpath+runid+'.'+str(years[ye])+'.oce.mean.nc'
        f       = Dataset(ncfile, 'r')    
        chl1     = f.variables[var_id1][dayind,:].mean(axis=0)
        chl2     = f.variables[var_id2][dayind,:].mean(axis=0)
        chl=chl1+chl2
        
     depth_list = [    0.,   -10.,   -20.,   -30.,   -40.,   -50.,   -60.,   -70.,
         -80.,   -90.,  -100.,  -115.,  -135.,  -160.,  -190.,  -230.,
        -280.,  -340.,  -410.,  -490.]
     level_data  = np.zeros((len(mesh.x2),len(depth_list)))
     for i in range(len(depth_list)):

        print 'Load data at '+str(depth_list[i])+'m'
        level_data1, elem_no_nan1 = pf.get_data(chl,mesh,depth_list[i])
        level_data[:,i]  = level_data1
        
     data_int=np.trapz(level_data,depth_list,axis=1)
     CHL[ye,:]=data_int
    data2=CHL.mean(axis=1)
    print 'Number of nans in tracer: ',np.count_nonzero(np.isnan(data2))
    print 'Number of inf in tracer: ',np.count_nonzero(np.isinf(data2))
    print 'Mean of surface: ',np.mean(data2[0:len(mesh.x2)])
    print 'Max and min: ',np.max(data2),np.min(data2)

    if export_csv == True:
        np.savetxt(outputpath+'CHLint_'+month+'_'+str(first_year)+'_'+str(last_year)+'_trend.csv', CHL, delimiter=";")
        np.savetxt(outputpath+'CHLint_'+month+'_'+str(first_year)+'_'+str(last_year)+'_mean.csv', data2, delimiter=";")
        print 'exporting done ...'

4
1990
Load data at 0.0m
For depth 0.0 model level 0.0 will be used
Load data at -10.0m
For depth -10.0 model level -10.0 will be used
Load data at -20.0m
For depth -20.0 model level -20.0 will be used
Load data at -30.0m
For depth -30.0 model level -30.0 will be used
Load data at -40.0m
For depth -40.0 model level -40.0 will be used
Load data at -50.0m
For depth -50.0 model level -50.0 will be used
Load data at -60.0m
For depth -60.0 model level -60.0 will be used
Load data at -70.0m
For depth -70.0 model level -70.0 will be used
Load data at -80.0m
For depth -80.0 model level -80.0 will be used
Load data at -90.0m
For depth -90.0 model level -90.0 will be used
Load data at -100.0m
For depth -100.0 model level -100.0 will be used
Load data at -115.0m
For depth -115.0 model level -115.0 will be used
Load data at -135.0m
For depth -135.0 model level -135.0 will be used
Load data at -160.0m
For depth -160.0 model level -160.0 will be used
Load data at -190.0m
For depth -190.0 model level

KeyboardInterrupt: 